In [1]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
import sys

sys.path.append('/private/groups/shapirolab/brock/Software')
from py3_functions import *

from IPython.display import display

!mkdir -p /private/groups/shapirolab/brock/mutation
os.chdir('/private/groups/shapirolab/brock/mutation')
!mkdir -p cmds logs

### R stuff
from rpy2 import rinterface
#from jupyter_helpers import rpy2_autocompletion
%load_ext rpy2.ipython

samples = []
with open('poplists/samples.txt') as infile:
    for line in infile:
        line = line.strip()
        samples.append(line)
infile.close()

chroms = []
with open('white_abalone.fasta.fai','r') as infile:
    for line in infile:
        chrom, size, total, b, c = line.split('\t')
        if int(size) > 1e6:
            chroms.append(chrom)
infile.close()

In [3]:
%%R
library(data.table)
library(tidyverse)
library(magrittr)
library(ggplot2)

# Repeats

Repeat modeler

In [15]:
!mkdir -p repeats
cmd = ('''BuildDatabase -name repeats/white_abalone '''
       '''white_abalone.fasta\n''').format()
slurm = make_slurm(id = 'builder', cmd_string = cmd, mem = '1000', time = '00:05:00', run = False, p = 'short')

In [4]:
cmd = ('''/private/groups/shapirolab/brock/Software/RepeatModeler-2.0.4/RepeatModeler '''
       '''-database repeats/white_abalone '''
       '''-threads 64 -LTRStruct '''
       '''--recoverDir RM_1215699.FriApr190928072024 '''
       '''>& logs/modeler.o\n''')
slurm = make_slurm(echo = False, id = 'modeler', c = 64, cmd_string = cmd, mem = '200000', time = '100:00:00', p = 'long')

RepeatMasker

In [5]:
!mkdir -p repeats
cmd = ('''RepeatMasker '''
       '''-q -lib repeats/white_abalone-families.fa '''
       '''-dir repeats '''
       '''-pa 16 '''
       '''-small -html -gff '''
       '''white_abalone.fasta > logs/masker.o 2>&1\n''').format()
slurm = make_slurm(id = f"masker",cmd_string = cmd, mem = '40000',time = '48:00:00',c='16',p='long')

# Mappability

First need to index genome

In [4]:
!mkdir -p mappability
cmd = ('''genmap index -F white_abalone.fasta -I mappability/index -v''')
print(cmd)

genmap index -F white_abalone.fasta -I mappability/index -v


This command runs very quick with `-E 0`

In [6]:
!mkdir -p mappability
cmd = ('''genmap map -K 150 -E 0 -I mappability/index -O mappability/150_by_0 -bg ''')
print(cmd)

genmap map -K 150 -E 0 -I mappability/index -O mappability/150_by_0 -bg 


In [9]:
%%bash
gsize=$(tail -n 1 white_abalone.fasta.fai | cut -f3)
echo $gsize

1184464861


In [24]:
%%R
gsize = 1184464861
genmap_0 = fread('mappability/150_by_0.bedgraph', showProgress = FALSE) %>% set_colnames(c('chrom','start','end','val')) %>% mutate(span = end - start)

## Proportion mappable with exact matches, 150 kmers
bases = genmap_0 %>% filter(val == 1.0) %>% pull(span) %>% sum
print(bases/gsize)

[1] 0.9390219


In reality, sequencing errors will occur and allow some degree of mismapping. So I create the following mask allowing up to 2 mismatches, with the goal of retaining regions ROBUST to reads with sequencing errors:

In [21]:
cmd = ('''genmap map -K 150 -E 2 -I mappability/index -O mappability/150_by_2 -bg ''')
print(cmd)

genmap map -K 150 -E 2 -I mappability/index -O mappability/150_by_2 -bg 


In [25]:
%%R
gsize = 1184464861
genmap_2 = fread('mappability/150_by_2.bedgraph', showProgress = FALSE) %>% set_colnames(c('chrom','start','end','val')) %>% mutate(span = end - start)

## Proportion mappable with up to 2 mismatches, 150 kmers
bases = genmap_2 %>% filter(val == 1.0) %>% pull(span) %>% sum
print(bases/gsize)

[1] 0.8955972


Now an extreme case with 4 mismatches. This runs a little slower

In [26]:
cmd = ('''genmap map -K 150 -E 4 -I mappability/index -O mappability/150_by_4 -bg ''')
print(cmd)

genmap map -K 150 -E 4 -I mappability/index -O mappability/150_by_4 -bg 


In [27]:
%%R
gsize = 1184464861
genmap_4 = fread('mappability/150_by_4.bedgraph', showProgress = FALSE) %>% set_colnames(c('chrom','start','end','val')) %>% mutate(span = end - start)

## Proportion mappable with up to 2 mismatches, 150 kmers
bases = genmap_4 %>% filter(val == 1.0) %>% pull(span) %>% sum
print(bases/gsize)

[1] 0.8620787


Bed formatting

In [14]:
%%bash
awk '{OFS="\t"} ($4 != 1) {print $1,$2,$3}' mappability/150_by_2.bedgraph | bedtools merge -i stdin > mappability/150_by_2_negative_mask.bed

# Sequencing depth masks

This code generates masks based on samtools depth output

Get mean, median, stdev, from `samtools depth` first chrom only (run off-screen)

In [ ]:
%%R
fs = list.files('depth/',pattern = 'mdup', full.names = T)
first_chrom_summary = bind_rows(lapply(fs,function(PATH){fread(PATH) %>% mutate(src = PATH) %>% group_by(src) %>% summarize(mean = mean(V3), stdev = sd(V3))}))
print(first_chrom_summary)

R[write to console]: |--------------------------------------------------|
|
R[write to console]: =====
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: =
R[write to console]: |

R[write to console]: |--------------------------------------------------|
|
R[write to console]: =====
R[write to console]: =
R[write to console]: ==
R[write to console]: ==
R[write to console]: =
R[write to console]: ==
R[write to co

Summarize for quick reading and use later on

In [36]:
%%R
first_chrom_summary %>%
    mutate(sample = basename(src) %>% gsub('_HiC_scaffold_1_mdup','',.)) %>%
    dplyr::select(!src) %>%
    relocate(sample) %>%
    fwrite('depth/depth_summary.txt',sep="\t",col.names=T)

## Masks from bam files

Get depth files for full genome for each sample. These are enormous

In [6]:
for SAMPLE in samples:
    if SAMPLE != 'NorbertW853': continue
    cmd = f"samtools depth -a -@ 8 aln/{SAMPLE}.mdup.bam > depth/{SAMPLE}.samtools.depth"
    slurm = make_slurm(run = False, id = f"{SAMPLE}.sm", cmd_string = cmd, c = 8, mem = '10000', time = '24:00:00', p = 'long')

The code below creates per-sample masks from these dpeth files, using 20 reads as the minimum (arguably conservative), and mean + (2*sd) as the upper limit. This takes a good second to run interactively

In [ ]:
sample_cov_cutoffs = {}
with open('depth/depth_summary.txt','r') as infile:
    next(infile)
    for line in infile:
        line = line.strip()
        sample, mean, stdev = line.split('\t')
        sample_cov_cutoffs[sample] = [mean,stdev]

bed_masks = {}
for SAMPLE, cutoffs in tqdm(sample_cov_cutoffs.items()):
    ##
    if SAMPLE != 'NorbertW853': continue
    mean,stdev = cutoffs
    upper = float(mean) + 2*float(stdev)
    path = f"depth/{SAMPLE}.samtools.depth"
    ##
    filtered_chunks = []
    for chunk in tqdm(pd.read_csv(path,sep="\t",header = None, chunksize=1000000)):
        chunk.columns = ['chrom','pos','depth']
        mk = chunk[(chunk['depth'] < 20) | (chunk['depth'] > upper)]
        filtered_chunks.append(mk)
    mask = pd.concat(filtered_chunks)
    mask.to_csv(f"depth/{SAMPLE}_bad_sites.txt",sep="\t",index = False, header = False)
    bed_masks[SAMPLE] = mask

The above code writes bad sites on a per-site basis. I'll quickly use bedtools to merge adjacent sites. In an ideal world, this would've been integrated into the code above.

In [5]:
%%bash
grep "NorbertW853" poplists/samples.txt | while read SAMPLE;do
    echo $SAMPLE;
    awk '{OFS="\t"} {print $1,$2-1,$2}' depth/${SAMPLE}_bad_sites.txt | bedtools merge -i stdin > depth/${SAMPLE}_bad_sites.bed;
done

NorbertW853


Extract only the 'total coverage' bad sites

In [29]:
%%bash
awk '{OFS="\t"} ($4 == 14) {print $1,$2,$3}' depth/common_bad_regions.bed > depth/all_masked.bed

Summarize over windows

In [59]:
%%bash
bedtools makewindows -g white_abalone.genome -w 10000 -s 10000 | bedtools coverage -a stdin -b depth/all_masked.bed > depth/all_masked_pct.bed

## Masks from allsites vcfs

In [7]:
for CHROM in chroms:
    cmd = f"bcftools query -f '%CHROM\\t%POS[\\t%DP]\\n' vcfs/{CHROM}.allsites.anno.vcf.gz | bgzip > depth/{CHROM}.vcf.depth.txt.gz "
    slurm = make_slurm(run = False, id = f"{CHROM}.vcfdepth",cmd_string = cmd, mem = '1000',time='03:00:00', p = 'long')

If we're going to be using info from the VCF to get depth, might as well summarize depth from the VCF as well:

In [ ]:
%%R
library(data.table)
library(tidyverse)
library(magrittr)
dp = fread('depth/HiC_scaffold_18.vcf.depth.txt.gz')
sample_order = fread(cmd = 'bcftools query -l vcfs/biallelic.vcf.gz', header = F) %>% pull(V1)
dp %<>% set_colnames(c('CHROM','POS',sample_order))

summary = 
    dp %>% pivot_longer(-c(CHROM,POS)) %>%
    group_by(name) %>% 
    summarize(mn = mean(value), sd = sd(value), median = median(value))

fwrite(summary,'depth/vcf_depth_summary.txt',col.names =T, quote = F, sep = "\t")

Now mask - similar to what was done with the bams above

In [ ]:
sample_cov_cutoffs = {}
with open('depth/vcf_depth_summary.txt','r') as infile:
    next(infile)
    for line in infile:
        line = line.strip()
        sample, mean, stdev, median = line.split('\t')
        sample_cov_cutoffs[sample] = [float(mean),float(stdev)]


filtered_chunks = {sample:[] for sample in samples}
# For a given chromosome
for CHROM in tqdm(chroms):
    path = f"depth/{CHROM}.vcf.depth.txt.gz"
    print(path)
    # For a chunk of the file
    for chunk in pd.read_csv(path,sep="\t",header = None, chunksize=10000000):
        chunk.columns = ['chrom','pos'] + samples
        
        # For each sample
        for SAMPLE in samples:
            upper = sample_cov_cutoffs[SAMPLE][0] + 2*sample_cov_cutoffs[SAMPLE][1]
            mk = chunk[(chunk[SAMPLE] < 10) | (chunk[SAMPLE] > upper)]
            filtered_chunks[SAMPLE].append(mk[['chrom','pos',SAMPLE]])

# Combine everything
masks = {sample:pd.concat(chunks) for sample,chunks in filtered_chunks.items()}



  0%|                                                                                                                                                           | 0/18 [00:00<?, ?it/s]

depth/HiC_scaffold_1.vcf.depth.txt.gz


  6%|████████▏                                                                                                                                          | 1/18 [01:37<27:45, 97.97s/it]

depth/HiC_scaffold_2.vcf.depth.txt.gz


 11%|████████████████▎                                                                                                                                  | 2/18 [03:05<24:32, 92.06s/it]

...

depth/HiC_scaffold_5.vcf.depth.txt.gz


 28%|████████████████████████████████████████▊                                                                                                          | 5/18 [07:11<18:07, 83.63s/it]

depth/HiC_scaffold_6.vcf.depth.txt.gz


 33%|█████████████████████████████████████████████████                                                                                                  | 6/18 [08:30<16:25, 82.16s/it]

depth/HiC_scaffold_7.vcf.depth.txt.gz


The below opearation uses pybedtools to merge adjacent intervals and take the coverage mean for that interval, to verify the right things are being masked. It takes QUITE A WHILE TO RUN which is annoying as hell

In [7]:
import pybedtools
merged = {}
for SAMPLE, DF in tqdm(masks.items()):
    DF['start'] = DF['pos'] - 1
    DF = DF[['chrom','start','pos',SAMPLE]]
    DF.columns = ['chrom','start','end','depth']
    g = pybedtools.BedTool.from_dataframe(DF)
    m = g.merge(c=4,o='mean').to_dataframe()
    merged[SAMPLE] = m


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [58:53<00:00, 252.40s/it]


Now write out the results

In [8]:
for SAMPLE, MASK in merged.items():
    MASK.to_csv(f"masks/{SAMPLE}_vcf_neg_mask.bed",sep="\t",index = False, header = False)

# Combine all
Finally, for each family, I combine the masks based on repeats, mappability, and depth to make a set of 'callable' regions that applies at the family level.

In [10]:
x = pd.read_csv('poplists/families.txt',sep="\t")
fam_2_samples = x.groupby('famName')['sampleName'].apply(list).to_dict()

!mkdir -p masks
for FAMILY, SAMPLELIST in fam_2_samples.items():
    bedstring = ''
    for SAMPLE in SAMPLELIST:
        ### Be careful with what bed files you are feeding in here
        bedstring += f"masks/{SAMPLE}_vcf_neg_mask_slop150.bed "

    ## MASKS BASED ON 
        #1) DEPTH
        #2) REPEATS
        #3) MAPPABILITY (REPETITIVE?)
    cmd = ('''cat {bedstring} mappability/150_by_2_negative_mask.bed <(cut -f1-3 repeats/white_abalone.fasta.out.bed) | bedtools sort -g white_abalone.genome -i stdin | bedtools merge -i stdin > masks/{FAMILY}_negative_mask.bed;'''
          '''bedtools complement -i masks/{FAMILY}_negative_mask.bed -g white_abalone.genome > masks/{FAMILY}_positive_mask.bed\n''').format(bedstring = bedstring, FAMILY = FAMILY)
    !$cmd
    cmd = ('''cat {bedstring} mappability/150_by_2_negative_mask.bed | bedtools sort -g white_abalone.genome -i stdin | bedtools merge -i stdin > masks/{FAMILY}_negative_REPINCLUDED_mask.bed;'''
          '''bedtools complement -i masks/{FAMILY}_negative_REPINCLUDED_mask.bed -g white_abalone.genome > masks/{FAMILY}_positive_REPINCLUDED_mask.bed\n''').format(bedstring = bedstring, FAMILY = FAMILY)
    !$cmd

Stats

In [18]:
%%R
library(data.table)
library(tidyverse)
library(ggplot2)
library(magrittr)
call_chroms = fread('call_chroms.txt')
gsize = sum(call_chroms$V2)
pos = 
    fread('masks/Fam1_positive_REPINCLUDED_mask.bed') %>% 
    set_colnames(c('chrom','start','end')) %>% 
    mutate(size = end - start + 1) %>%
    filter(chrom %in% call_chroms$V1)
print('Pct usable genome for Fam 1:')
print(sum(pos$size))
print(sum(pos$size)/gsize)

pos = 
    fread('masks/Fam2_positive_REPINCLUDED_mask.bed') %>% 
    set_colnames(c('chrom','start','end')) %>% 
    mutate(size = end - start + 1) %>%
    filter(chrom %in% call_chroms$V1)
print('Pct usable genome for Fam 2:')
print(sum(pos$size))
print(sum(pos$size)/gsize)


pos = 
    fread('masks/Fam3_positive_REPINCLUDED_mask.bed') %>% 
    set_colnames(c('chrom','start','end')) %>% 
    mutate(size = end - start + 1) %>%
    filter(chrom %in% call_chroms$V1)
print('Pct usable genome for Fam 3:')
print(sum(pos$size))
print(sum(pos$size)/gsize)



[1] "Pct usable genome for Fam 1:"
[1] 891434842
[1] 0.7680365
[1] "Pct usable genome for Fam 2:"
[1] 898249068
[1] 0.7739074
[1] "Pct usable genome for Fam 3:"
[1] 917622025
[1] 0.7905986
